<h1>Getting Started</h1>

In [5]:
# if you are running in the notebooks folder example cloned from github this line must be included
import sys
import os
sys.path.insert(0, os.path.abspath('../'))

<h3>Import CtllDes and poliastro, the core and utils elements </h3>

In [6]:
import numpy as np

#Propagator engine
import poliastro

#The library
import CtllDes 
from CtllDes.core import ctll, satellite

#Throughout this examples it will be necessary to use the correct
#physical dimensions, astropy.units will help us with that
import astropy.units as u

# Building an orbit 

The first step is to define an orbit. You can do this in a variety of ways as showed below.

### Building from classical orbit parameters

In [7]:
#Orbit object, more info on poliastro API reference.
from poliastro.twobody import Orbit

#The bodies module specifies attractors
from poliastro.bodies import Earth


#Planes of reference and Epochs, are almost every time 
#the default ones, but just in case if you want to use 
#any other, they can be found in the following modules

#import planes of reference, EARTH_EQUATOR 
from poliastro.frames import Planes

#import J2000 time reference from constants 
from poliastro.constants import J2000



### Specify classical orbit parameters ###

a = 8000 * u.km # semi-major axis [distance]
ecc = 0 * u.one # eccentricity [dimensionleess]
inc = 98 * u.deg # inclination [angle] 
raan = 0 * u.rad # right ascencion of the ascending node [angle]
argp = 0 * u.rad # perigee argument [angle]
nu = 0 * u.rad # true anomaly [angle]

plane = Planes.EARTH_EQUATOR # not necessary to specify
epoch = J2000 # not necessary to specify

#classmethod of Orbit
orb = Orbit.from_classical(Earth,
                            a,
                            ecc,
                            inc,
                            raan,
                            argp,
                            nu)


### Other poliastro.twobody.Orbit classmethods

In [8]:
#more on poliastro API reference
orb = Orbit.heliosynchronous(Earth, a=8000*u.km, ecc=0*u.one)
orb = Orbit.from_vectors(Earth, [8000,0,0]*u.km, [0,7,0]*u.km/u.s)

### Orbit utilities for repeating ground track and heliosynchronous orbits

In [9]:
from CtllDes.utils import sscalc

D = 7 #days of revisit
orb_df = sscalc.helio_rgt_synchronous(D) #dataframe
orb_df #radius [km], inclination [degrees], revolutions per day.

,radius,inc,N
0,11544.182918,142.248541,7.000000
1,11389.626716,138.961424,7.142857
2,11240.148578,136.071863,7.285714
3,11095.486092,133.496401,7.428571
4,10955.395204,131.177130,7.571429
...,...,...,...
66,6535.898060,96.204172,16.428571
67,6498.279346,96.079672,16.571429
68,6461.197311,95.958727,16.714286
69,6424.639809,95.841203,16.857143


In [10]:
#use pandas to filter the desired parameters
earth_R = Earth.R_mean.to(u.km).value #value in km of earth mean Radius

min_radius = 500 + earth_R
max_radius = 900 + earth_R
filtered = orb_df.loc[(orb_df['radius'].values < max_radius) &
                      (orb_df['radius'].values > min_radius)]
filtered 

,radius,inc,N
50,7222.422363,98.816646,14.142857
51,7174.189097,98.610796,14.285714
52,7126.753168,98.411801,14.428571
53,7080.093692,98.219362,14.571429
54,7034.190529,98.033200,14.714286
55,6989.024251,97.853046,14.857143
56,6944.576109,97.678649,15.000000
57,6900.828006,97.509769,15.142857


In [11]:
#select the desired cop, i.e. classical orbit parameters
cop = filtered.loc[55] #pick one from the df
a = cop['radius']*u.km #radius = semi-major axis
ecc = 0 * u.one # this is cant be different than 0 since the helio_rgt is for circular orbits
inc = cop['inc']*u.deg

#reuse past raan, argp and nu
orb = Orbit.from_classical(Earth,
                            a,
                            ecc,
                            inc,
                            raan,
                            argp,
                            nu)

# Building satellites

 Satellite is one of the main classes in this library. You can build a satellite reading the docs, you are encouraged to use the classmethods.


In [12]:
from CtllDes.core.satellite import Sat


### Building satellite from classmethods.

In [13]:
sat = Sat.from_orbit(orb) 
sat = Sat.from_vectors([8000,0,0]*u.km,
                       [0,7,0]*u.km/u.s,
                      attractor=Earth)


# Satellite's parameters 

In [14]:
sat.info() #general info at t=0

id : 8cb757cf-9a8b-4fa5-b63b-60680cbda73f	[a ecc inc raan argp nu] : [7869.7 km 0.0 0.0 rad 0.0 rad 3.1 rad -3.1 rad]		status : Online


### Physical Specifications

In [15]:
print(sat.spec) #None specified, default parameters selected

 m:1.0 kg, Cd:1.0, A:1.0 m2


### Status

In [16]:
sat.status #None specified, Online status 

'Online'

### Instruments

In [17]:
sat.instruments #None specified, empty list

[]

### Universal Unique Identfier


In [18]:
sat.id

UUID('8cb757cf-9a8b-4fa5-b63b-60680cbda73f')

### Attractor

In [19]:
sat.attractor

Earth (♁)

### Orbit

In [20]:
sat.orbit

7739 x 8000 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)

#  Physical Specifications

In [21]:
from CtllDes.core import specs

In [22]:

drag_coef = 2.2*u.one
mass = 200*u.kg
Area = 10*u.m**2

spec = specs.Specifications(m=mass, Cd=drag_coef, A=Area)

In [23]:
sat.update_spec(spec)
print(sat.spec)

 m:200.0 kg, Cd:2.2, A:10.0 m2


#  Status (in dev)

In [24]:
#In development. Until now is just a string identifying if the 
#satellite is Online or not. This will impact when calculating
#merit figures. 

In [25]:
sat.update_status('Shutoff')
sat.status

Warning, notImplemented


'Online'

# Instruments

The instruments module is explained in more detailed in instruments.ipynb. There you can find how to create new instruments and For now, the example below will just update the instrument with a default Camera one. 

In [26]:
from CtllDes.core import instrument

In [27]:
camera = instrument.Camera(800, 0.01) # pixels, angle resolution
sat.update_instruments(camera,f=True) # force option
sat.instruments

# Building a Constellation

In [28]:
#since a constellation is a set of satellite we will build a few more.

sat0 = sat
sat1 = Sat.from_vectors([8000,0,0]*u.km,
                        [0,7,0]*u.km/u.s,
                        attractor=Earth)
sat2 = Sat.from_orbit(Orbit.heliosynchronous(Earth,
                                             a=8000*u.km,
                                             ecc=0*u.one))


In [29]:
constellation = ctll.Ctll.from_sats([sat0,sat1,sat2])

In [30]:
constellation.info()


3 satellites within no Pattern


In [31]:
constellation.info(v=True)


3 satellites in no Pattern

id : b1b7e743-a13e-40b4-bbe6-df258e79715b	[a ecc inc raan argp nu] : [7869.7 km 0.0 0.0 rad 0.0 rad 3.1 rad -3.1 rad]		status : Online
id : 5668e285-2298-46bd-a8d4-7a6f76714a30	[a ecc inc raan argp nu] : [7869.7 km 0.0 0.0 rad 0.0 rad 3.1 rad -3.1 rad]		status : Online
id : ce95bc3d-6ab2-4365-b5e3-4323812a0ff7	[a ecc inc raan argp nu] : [8000.0 km 0.0 1.8 rad 610.5 deg 0.0 deg 0.0 deg]		status : Online


In [32]:
ids = constellation.online_id
ids 

[UUID('b1b7e743-a13e-40b4-bbe6-df258e79715b'),
 UUID('5668e285-2298-46bd-a8d4-7a6f76714a30'),
 UUID('ce95bc3d-6ab2-4365-b5e3-4323812a0ff7')]

In [33]:
sat_by_id = constellation.get_sat(ids[1])
sat_by_id.info()

id : 5668e285-2298-46bd-a8d4-7a6f76714a30	[a ecc inc raan argp nu] : [7869.7 km 0.0 0.0 rad 0.0 rad 3.1 rad -3.1 rad]		status : Online


In [34]:
constellation.sats[1].info()

id : 5668e285-2298-46bd-a8d4-7a6f76714a30	[a ecc inc raan argp nu] : [7869.7 km 0.0 0.0 rad 0.0 rad 3.1 rad -3.1 rad]		status : Online


In [35]:
constellation.instruments

[[<CtllDes.core.instrument.Camera at 0x7fd94a011310>], [], []]

In [36]:
camera1 = instrument.Camera(800,0.1)
camera2 = instrument.Camera(800,0.04)

In [37]:
#this wouldnt work since the constellation copies the sat info
sat1.update_instruments(camera1,f=True)
sat2.update_instruments(camera2,f=True)

constellation.instruments

[[<CtllDes.core.instrument.Camera at 0x7fd94a011310>], [], []]

In [38]:
constellation.sats[1].update_instruments(camera1,f=True)
constellation.sats[2].update_instruments(camera2,f=True)

constellation.instruments

[[<CtllDes.core.instrument.Camera at 0x7fd94a011310>],

### Building specific pattern 

In [39]:
#Build a Walker Delta constellation
T = 10
P = 5
F = 3

p = 12000*u.km
ecc = 0.3*u.one
inc = 99*u.deg
raan = 30*u.deg
argp = 20*u.deg
nu = 0.3*u.rad

WDctll = ctll.Ctll.from_WalkerDelta(T,
                                    P,
                                    F,
                                    p,
                                    ecc,
                                    inc,
                                    argp,
                                    raan_offset=raan,
                                    nu_offset=nu,
                                   )

In [40]:
WDctll.info(v=True)


10 satellites in Walker Delta Pattern T/P/F = 10/5/3

id : bc5f88e1-1cfd-4a05-b1b8-d3a738157a14	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 30.0 deg 20.0 deg 17.2 deg]		status : Online
id : 46dcf50f-35c3-4ee7-8eac-946e2ab93500	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 30.0 deg 20.0 deg -162.8 deg]		status : Online
id : 4c1caa67-7c6d-4e4c-b6fe-f052e58e5fd8	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 102.0 deg 20.0 deg 125.2 deg]		status : Online
id : 6f848a57-8079-48c3-9446-2446882f38e9	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 102.0 deg 20.0 deg -54.8 deg]		status : Online
id : a5f1e4c4-3c28-4750-a16f-1ff77f334ece	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 174.0 deg 20.0 deg -126.8 deg]		status : Online
id : b298a7ae-abd1-4a26-9a1c-2801ee1220ba	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 174.0 deg 20.0 deg 53.2 deg]		status : Online
id : ebde6ae6-b133-4733-95c2-21c1a73fa20f	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 246.0 d

In [41]:
#default specs
print("\nSpecifications\n")
for spec in WDctll.specs:
    print(spec)

#default status
print("\nStatus\n")
for status in WDctll.status:
    print(status)

#default instruments
print("\nInstruments\n")
for instr in WDctll.instruments:
    print(instr)



Specifications

 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2
 m:1.0 kg, Cd:1.0, A:1.0 m2

Status

Online
Online
Online
Online
Online
Online
Online
Online
Online
Online

Instruments

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


# Changing default instruments, status and specifications.

In [42]:
#check amount of satellites inside Walker Delta Ctll
WDctll.N

10

### Specifications

In [43]:
general_spec = specs.Specifications(m=30*u.kg,Cd=1.5*u.one,A=50*u.m**2)
spec_lst = [general_spec for _ in range(WDctll.N)]
WDctll.update_specs(spec_lst)

#new specs
for spec in WDctll.specs:
    print(spec)

 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2
 m:30.0 kg, Cd:1.5, A:50.0 m2


### Instruments

In [44]:
general_instrument = instrument.Camera(1000,0.1)
instr_lst = [general_instrument for _ in range(WDctll.N)]
WDctll.update_instruments(instr_lst,f=True)

#new instruments
for instr in WDctll.instruments:
    print(instr)

### Status (in dev)

In [45]:
#

# Adding Constellations  

In [46]:
added_ctll = constellation + WDctll
added_ctll.info(v=True)



10 satellites in Walker Delta Pattern T/P/F = 10/5/3

id : 4606a4f5-6658-4314-adbc-610dacc0192c	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 30.0 deg 20.0 deg 17.2 deg]		status : Online
id : 3a05a2e7-a58d-4737-b253-1422af418001	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 30.0 deg 20.0 deg -162.8 deg]		status : Online
id : f3717400-bb8f-421d-ba84-42d84c0ed5e3	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 102.0 deg 20.0 deg 125.2 deg]		status : Online
id : a5f238a1-8a94-4b8d-8466-075ede948846	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 102.0 deg 20.0 deg -54.8 deg]		status : Online
id : 1a9ce704-e69f-4dea-9391-c41ec2940e08	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 174.0 deg 20.0 deg -126.8 deg]		status : Online
id : 6bebfa48-9109-40be-8224-7579ed6f2957	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 174.0 deg 20.0 deg 53.2 deg]		status : Online
id : 46dac602-87ad-4db5-8bcf-42e013604494	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 246.0 d

In [47]:
#the above cell is equivalent to the following except from the fact that the patterns are not inherited
added_ctll = ctll.Ctll.from_sats(constellation.sats+WDctll.sats)
added_ctll.info(v=True)


13 satellites in no Pattern

id : b630a2b9-bb8a-43bd-b61d-89d636bd904a	[a ecc inc raan argp nu] : [7869.7 km 0.0 0.0 rad 0.0 rad 3.1 rad -3.1 rad]		status : Online
id : 55873648-ed29-4ecc-99a5-f9061f4fdf4f	[a ecc inc raan argp nu] : [7869.7 km 0.0 0.0 rad 0.0 rad 3.1 rad -3.1 rad]		status : Online
id : c8a744a7-ca0a-4c01-b0b4-930c0d9da1bb	[a ecc inc raan argp nu] : [8000.0 km 0.0 1.8 rad 610.5 deg 0.0 deg 0.0 deg]		status : Online
id : 0eb119cb-2c0d-4ab1-b6f7-97becd705fa9	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 30.0 deg 20.0 deg 17.2 deg]		status : Online
id : 4ca6baa5-58b0-483a-be38-0d24674f34fd	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 30.0 deg 20.0 deg -162.8 deg]		status : Online
id : 00e1cdc8-bc72-4b15-97d4-653e7dd210cf	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 102.0 deg 20.0 deg 125.2 deg]		status : Online
id : 014d1452-396d-4743-8076-fb1286e2072e	[a ecc inc raan argp nu] : [13186.8 km 0.3 99.0 deg 102.0 deg 20.0 deg -54.8 deg]		status : Online
id

# Propagation

In order to obtain the ephemerides of the satellite you can use the rv method for the Sat class as well as the Ctll class.

### Satellite

In [48]:
#As you can see in the documentation 

#T days 
#dt interval in seconds
#drag and J2, perturbations used in propagation
r_sat,v_sat = added_ctll.sats[0].rv(30,dt=50, J2=True, drag=False)
lon_sat,lat_sat = added_ctll.sats[0].ssps(4,J2=False)

### Constellation

In [49]:
rv_ctll = added_ctll.rv(30, dt=10, drag = True, J2=True)
r,v = rv_ctll[0]

lonlat_ctll = added_ctll.ssps(5,dt=10,J2=True,drag=True)
lon,lat = lonlat_ctll[0]

### Plotting r (xyz)

In [50]:
import matplotlib.pyplot as plt
%matplotlib qt5

In [51]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(r_sat[::50,0], r_sat[::50,1], r_sat[::50,2],s=0.5,c='k') #xyz obviously
ax.scatter(0,0,0,c='blue', s=6370)

plt.show()


In [52]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(r[::50,0], r[::50,1], r[::50,2],s=0.5,c='k') #xyz obviously
ax.scatter(0,0,0,c='blue',s=6370)

plt.show()

### Plotting groundtrack 

In [53]:
lons, lats = added_ctll.sats[-1].ssps(1,J2=True,drag=False)

fig = plt.figure()
plt.scatter(lons[::10],lats[::10],c='k',s=0.1)
plt.show()
 

array([1.04719755, 1.23095942])